2025-04-23 14:24:29,337 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-04-23 14:24:29,340 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-23 14:24:29,535 INFO sqlalchemy.engine.Engine select current_schema()
2025-04-23 14:24:29,537 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-23 14:24:29,738 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-04-23 14:24:29,739 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-23 14:24:29,949 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-23 14:24:29,968 INFO sqlalchemy.engine.Engine SELECT max(blocks.datetime) AS max_datetime 
FROM blocks 
WHERE blocks.chain_id = %(chain_id_1)s
2025-04-23 14:24:29,968 INFO sqlalchemy.engine.Engine [generated in 0.00055s] {'chain_id_1': 1}
2025-04-23 14:24:30,190 INFO sqlalchemy.engine.Engine COMMIT
1726365887 highest_timestamp
adding [1726444799, 1726531199, 1726617599, 1726703999, 1726790399, 1726876799, 1726963199, 1727049599, 1727135999, 1727222399, 1727308799, 1727

,event,block,transaction_index,log_index,hash,vaultAddress,caller
0,DestinationVaultRegistered,20729597,102,272,0x80517c0004ce71c9fa6e9b745eaa36846476f1286ccf...,0x1E581AD9ef141C3037991806383B7990c1f68312,0xC6c32cd2197dd6badd828051E50664E3cB199678
1,DestinationVaultRegistered,20729598,52,92,0x0304e0754ab8807bc5d8a789c2ba8343a6754fab4a01...,0x013D82483C968cdbB4a8384754f5a5028b85dB6d,0xC6c32cd2197dd6badd828051E50664E3cB199678
2,DestinationVaultRegistered,20729599,118,367,0x8d1ed3ec3da61e993d481c986058983844fff9ad2d76...,0x84844d8A8a13337b31CF01DC384c74C862FCd903,0xC6c32cd2197dd6badd828051E50664E3cB199678
3,DestinationVaultRegistered,20729600,132,312,0x9959a356527ecfce4978d401e1467f73cf9a9ca3e997...,0x003aD6a060a1082B7A6F85d24dDF9E9E196B8426,0xC6c32cd2197dd6badd828051E50664E3cB199678
4,DestinationVaultRegistered,20729601,54,145,0x8b0551e4786ffa8222cdf9a8d4144b1a9cbba8494388...,0x4F4bE87cFe874db2e248A787669a0D6E77f1F149,0xC6c32cd2197dd6badd828051E50664E3cB199678
...,...,...,...,...,...,...,...
97,DestinationVaultRegistered,22196277,12,102,0x47d6011f13c5e6a1087eb4b749ef26edc42415e12762...,0x7677569e823D24c15050f7d3d036Ba053785066b,0x37De6f9D23d26DfbCB6dbEE86Dc7ca538c0ff124
98,DestinationVaultRegistered,22297038,195,358,0x925f31e6d77ede4d3e5a8ab61c5f7cdea37b5625cbd7...,0xAd7b8dB1efEe849c67060c6b10BC7D6D3C607aa5,0x37De6f9D23d26DfbCB6dbEE86Dc7ca538c0ff124
99,DestinationVaultRegistered,22297039,197,291,0xfc95f747e3dbdd53e1c53362544d12dae211574b9c89...,0x6D9b1cF3698Fd5f2E2F5E6344A20A4c062960bC9,0x37De6f9D23d26DfbCB6dbEE86Dc7ca538c0ff124
100,DestinationVaultRegistered,22297344,101,384,0x8b73b332f59c4de58c16a030acd1e988cdc1a6d93961...,0x7876F91BB22148345b3De16af9448081E9853830,0x37De6f9D23d26DfbCB6dbEE86Dc7ca538c0ff124


In [16]:
from mainnet_launch.constants import DESTINATION_VAULT_REGISTRY, WETH, ETH_CHAIN, eth_client, ChainData
from mainnet_launch.abis import DESTINATION_VAULT_REGISTRY_ABI
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.data_fetching.get_state_by_block import get_state_by_one_block, identity_with_bool_success
from mainnet_launch.database.schema.full import Destinations, DestinationTokens
from multicall import Call, Multicall
from web3 import Web3


def _fetch_new_destinations_from_external_source(chain: ChainData, highest_block_already_found: int):
    contract = eth_client.eth.contract(DESTINATION_VAULT_REGISTRY(chain), abi=DESTINATION_VAULT_REGISTRY_ABI)
    DestinationVaultRegistered = fetch_events(
        contract.events.DestinationVaultRegistered, start_block=highest_block_already_found, end_block=None, chain=chain
    )
    highest_block = int(DestinationVaultRegistered["block"].max())

    vaults = DestinationVaultRegistered["vaultAddress"]
    # block == block
    symbol_calls = [
        Call(
            v,
            "symbol()(string)",
            [(v + "_symbol", identity_with_bool_success)],
        )
        for v in vaults
    ]

    name_calls = [
        Call(
            v,
            "name()(string)",
            [(v + "_name", identity_with_bool_success)],
        )
        for v in vaults
    ]

    pool_type_calls = [
        Call(
            v,
            "poolType()(string)",
            [(v + "_pool_type", identity_with_bool_success)],
        )
        for v in vaults
    ]

    exchange_name_calls = [
        Call(
            v,
            "exchangeName()(string)",
            [(v + "_exchange_name", identity_with_bool_success)],
        )
        for v in vaults
    ]

    underlying_calls = [
        Call(
            v,
            "underlying()(address)",
            [(v + "_underlying", identity_with_bool_success)],
        )
        for v in vaults
    ]

    pool_calls = [
        Call(
            v,
            "getPool()(address)",
            [(v + "_pool", identity_with_bool_success)],
        )
        for v in vaults
    ]

    underlying_tokens_calls = [
        Call(
            v,
            "underlyingTokens()(address[])",
            [(v + "_underlyingTokens", identity_with_bool_success)],
        )
        for v in vaults
    ]
    calls = [
        *symbol_calls,
        *name_calls,
        *pool_type_calls,
        *exchange_name_calls,
        *underlying_calls,
        *pool_calls,
        *underlying_tokens_calls,
    ]

    raw = get_state_by_one_block(calls, highest_block, chain)
    # vaults = DestinationVaultRegistered["vaultAddress"]

    # one dict per attribute
    symbol_dict = {v: raw[f"{v}_symbol"] for v in vaults}
    name_dict = {v: raw[f"{v}_name"] for v in vaults}
    pool_type_dict = {v: raw[f"{v}_pool_type"] for v in vaults}
    exchange_name_dict = {v: raw[f"{v}_exchange_name"] for v in vaults}
    underlying_dict = {v: raw[f"{v}_underlying"] for v in vaults}
    pool_dict = {v: raw[f"{v}_pool"] for v in vaults}
    underlying_tokens_dict = {v: raw[f"{v}_underlyingTokens"] for v in vaults}

    underlying_symbol_calls = [
        Call(
            underlying_dict[v],
            "symbol()(string)",
            [(v + "_underlying_symbol", identity_with_bool_success)],
        )
        for v in vaults
    ]

    underlying_names_calls = [
        Call(
            underlying_dict[v],
            "symbol()(string)",
            [(v + "_underlying_name", identity_with_bool_success)],
        )
        for v in vaults
    ]

    raw = get_state_by_one_block([*underlying_symbol_calls, *underlying_names_calls], highest_block, ETH_CHAIN)
    # vaults = DestinationVaultRegistered["vaultAddress"]

    # one dict per attribute
    underlying_symbol_dict = {v: raw[f"{v}_underlying_symbol"] for v in vaults}
    underlying_name_dict = {v: raw[f"{v}_underlying_name"] for v in vaults}
    vault_to_block = {
        v: b for v, b in zip(DestinationVaultRegistered["vaultAddress"], DestinationVaultRegistered["block"])
    }

    destinations = [
        Destinations(
            destination_vault_address=v,
            chain_id=ETH_CHAIN.chain_id,
            exchange_name=exchange_name_dict[v],
            block_deployed=vault_to_block[v],
            name=name_dict[v],
            symbol=symbol_dict[v],
            pool_type=pool_type_dict[v],
            pool=pool_dict[v],
            underlying=underlying_dict[v],
            underlying_symbol=underlying_symbol_dict[v],
            underlying_name=underlying_name_dict[v],
        )
        for v in vaults
    ]

    # destination_tokens = []
    # for v in vaults:
    #     for index, token_address in enumerate(underlying_tokens_dict[v]):
    #         destination_tokens.append(
    #             DestinationTokens(
    #                 destination_vault_address=v, chain_id=chain.chain_id, token_address=token_address, index=index
    #             )
    #         )

    # return destinations, destination_tokens


destinations, destination_tokens = _fetch_new_destinations_from_external_source(ETH_CHAIN, 18_000_000)

2025-04-23 14:36:25,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-23 14:36:25,224 INFO sqlalchemy.engine.Engine SELECT max(blocks.datetime) AS max_datetime 
FROM blocks 
WHERE blocks.chain_id = %(chain_id_1)s
2025-04-23 14:36:25,225 INFO sqlalchemy.engine.Engine [cached since 715.3s ago] {'chain_id_1': 1}
2025-04-23 14:36:25,418 INFO sqlalchemy.engine.Engine COMMIT
2025-04-23 14:36:25,515 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-23 14:36:25,516 INFO sqlalchemy.engine.Engine SELECT max(blocks.block) AS block 
FROM blocks 
WHERE blocks.chain_id = %(chain_id_1)s AND blocks.block >= %(block_1)s AND blocks.block <= %(block_2)s GROUP BY date_trunc(%(date_trunc_1)s, blocks.datetime) ORDER BY date_trunc(%(date_trunc_2)s, blocks.datetime)
2025-04-23 14:36:25,516 INFO sqlalchemy.engine.Engine [cached since 705.9s ago] {'chain_id_1': 1, 'block_1': 20752910, 'block_2': 100000000, 'date_trunc_1': 'day', 'date_trunc_2': 'day'}
2025-04-23 14:36:25,707 INFO sqlalchemy.engine

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
from web3 import Web3

vaults = DestinationVaultRegistered["vaultAddress"]
# block == block
symbol_calls = [
    Call(
        v,
        "symbol()(string)",
        [(v + "_symbol", identity_with_bool_success)],
    )
    for v in vaults
]

name_calls = [
    Call(
        v,
        "name()(string)",
        [(v + "_name", identity_with_bool_success)],
    )
    for v in vaults
]

pool_type_calls = [
    Call(
        v,
        "poolType()(string)",
        [(v + "_pool_type", identity_with_bool_success)],
    )
    for v in vaults
]

exchange_name_calls = [
    Call(
        v,
        "exchangeName()(string)",
        [(v + "_exchange_name", identity_with_bool_success)],
    )
    for v in vaults
]


underlying_calls = [
    Call(
        v,
        "underlying()(address)",
        [(v + "_underlying", identity_with_bool_success)],
    )
    for v in vaults
]

pool_calls = [
    Call(
        v,
        "getPool()(address)",
        [(v + "_pool", identity_with_bool_success)],
    )
    for v in vaults
]

underlying_tokens_calls = [
    Call(
        v,
        "underlyingTokens()(address[])",
        [(v + "_underlyingTokens", identity_with_bool_success)],
    )
    for v in vaults
]
calls = [
    *symbol_calls,
    *name_calls,
    *pool_type_calls,
    *exchange_name_calls,
    *underlying_calls,
    *pool_calls,
    *underlying_tokens_calls,
]

raw = get_state_by_one_block(calls, 22328135, ETH_CHAIN)
# vaults = DestinationVaultRegistered["vaultAddress"]

# one dict per attribute
symbol_dict = {v: raw[f"{v}_symbol"] for v in vaults}
name_dict = {v: raw[f"{v}_name"] for v in vaults}
pool_type_dict = {v: raw[f"{v}_pool_type"] for v in vaults}
exchange_name_dict = {v: raw[f"{v}_exchange_name"] for v in vaults}
underlying_dict = {v: raw[f"{v}_underlying"] for v in vaults}
pool_dict = {v: raw[f"{v}_pool"] for v in vaults}
underlying_tokens_dict = {v: raw[f"{v}_underlyingTokens"] for v in vaults}

underlying_symbol_calls = [
    Call(
        underlying_dict[v],
        "symbol()(string)",
        [(v + "_underlying_symbol", identity_with_bool_success)],
    )
    for v in vaults
]

underlying_names_calls = [
    Call(
        underlying_dict[v],
        "symbol()(string)",
        [(v + "_underlying_name", identity_with_bool_success)],
    )
    for v in vaults
]


raw = get_state_by_one_block([*underlying_symbol_calls, *underlying_names_calls], 22328135, ETH_CHAIN)
# vaults = DestinationVaultRegistered["vaultAddress"]

# one dict per attribute
underlying_symbol_dict = {v: raw[f"{v}_underlying_symbol"] for v in vaults}
underlying_name_dict = {v: raw[f"{v}_underlying_name"] for v in vaults}
vault_to_block = {v: b for v, b in zip(DestinationVaultRegistered["vaultAddress"], DestinationVaultRegistered["block"])}


destinations = [
    Destinations(
        destination_vault_address=v,
        chain_id=ETH_CHAIN.chain_id,
        exchange_name=exchange_name_dict[v],
        block_deployed=vault_to_block[v],
        name=name_dict[v],
        symbol=symbol_dict[v],
        pool_type=pool_type_dict[v],
        pool=pool_dict[v],
        underlying=underlying_dict[v],
        underlying_symbol=underlying_symbol_dict[v],
        underlying_name=underlying_name_dict[v],
    )
    for v in vaults
]
destinations

[Destinations(destination_vault_address='0x1E581AD9ef141C3037991806383B7990c1f68312', chain_id=1, exchange_name='balancer', block_deployed=20729597, name='Tokemak-Wrapped Ether-Balancer rETH Stable Pool', symbol='toke-WETH-B-rETH-STABLE', pool_type='balMetaStable', pool='0x1e19cf2d73a72ef1332c882f20534b6519be0276', underlying='0x1e19cf2d73a72ef1332c882f20534b6519be0276', underlying_symbol='B-rETH-STABLE', underlying_name='B-rETH-STABLE'),
 Destinations(destination_vault_address='0x013D82483C968cdbB4a8384754f5a5028b85dB6d', chain_id=1, exchange_name='balancer', block_deployed=20729598, name='Tokemak-Wrapped Ether-Balancer osETH/wETH StablePool', symbol='toke-WETH-osETH/wETH-BPT', pool_type='balCompStable', pool='0xdacf5fa19b1f720111609043ac67a9818262850c', underlying='0xdacf5fa19b1f720111609043ac67a9818262850c', underlying_symbol='osETH/wETH-BPT', underlying_name='osETH/wETH-BPT'),
 Destinations(destination_vault_address='0x84844d8A8a13337b31CF01DC384c74C862FCd903', chain_id=1, exchange

In [4]:
for k, v in a.items():
    print(k, v)

NameError: name 'a' is not defined

In [ ]:
# from sqlalchemy_schemadisplay import create_schema_graph
# from mainnet_launch.database.schema.full import Base

# # Build the graph from your ORM metadata
# graph = create_schema_graph(
#     metadata=Base.metadata,
#     show_datatypes=False,    # you can turn this on if you like
#     show_indexes=False,
#     rankdir='LR'             # left-to-right layout
# )

# # Write out to a PNG (or .pdf, .svg, etc)
# graph.write_png('schema.png')
# print("Wrote schema.png")